Generate a business idea with Amazon Bedrock.


In [3]:
# Install necessary packages
!uv add boto3

Resolved 222 packages in 1.30s                                       
⠙ Preparing packages... (0/4)                                                   
⠙ Preparing packages... (0/4)--------------     0 B/83.31 KiB           
⠙ Preparing packages... (0/4)--------------     0 B/83.31 KiB           
jmespath             ------------------------------     0 B/19.78 KiB
⠙ Preparing packages... (0/4)--------------     0 B/83.31 KiB           
jmespath             ------------------------------     0 B/19.78 KiB
⠙ Preparing packages... (0/4)-------------- 14.88 KiB/83.31 KiB         
jmespath             ------------------------------ 14.89 KiB/19.78 KiB
⠙ Preparing packages... (0/4)-------------- 14.88 KiB/83.31 KiB         
jmespath             ------------------------------ 14.89 KiB/19.78 KiB
⠙ Preparing packages... (0/4)-------------- 14.88 KiB/83.31 KiB         
jmespath             ------------------------------ 14.89 KiB/19.78 KiB
⠙ Preparing packages... (0/4)-------------- 14.88 KiB/8

In [17]:
import boto3
import os
from dotenv import load_dotenv
from time import sleep


In [20]:
# Load api key from .env or environment variable. This notebook is using the simpler API key method, which gives access only to Amazon Bedrock services, instead of standard AWS credentials

load_dotenv(override=True)

os.environ['AWS_BEARER_TOKEN_BEDROCK'] = os.getenv('AWS_BEARER_TOKEN_BEDROCK', 'your-key-if-not-using-env')

region = 'us-east-1' # change to your preferred region - be aware that not all regions have access to all models. If in doubt, use us-east-1.

bedrock = boto3.client(service_name="bedrock", region_name=region) # use this for information and management calls (such as model listings)
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name=region) # this is for inference.


In [21]:
# Let's do a quick test to see if works.
# We will list the available models.

response = bedrock.list_foundation_models()
models = response['modelSummaries']
print(f'AWS Region: {region} - Models:')
for model in models:
    print(f"Model ID: {model['modelId']}, Name: {model['modelName']}")

AWS Region: us-east-1 - Models:
Model ID: twelvelabs.pegasus-1-2-v1:0, Name: Pegasus v1.2
Model ID: anthropic.claude-opus-4-1-20250805-v1:0, Name: Claude Opus 4.1
Model ID: amazon.titan-tg1-large, Name: Titan Text Large
Model ID: amazon.titan-image-generator-v1:0, Name: Titan Image Generator G1
Model ID: amazon.titan-image-generator-v1, Name: Titan Image Generator G1
Model ID: amazon.titan-image-generator-v2:0, Name: Titan Image Generator G1 v2
Model ID: amazon.nova-premier-v1:0:8k, Name: Nova Premier
Model ID: amazon.nova-premier-v1:0:20k, Name: Nova Premier
Model ID: amazon.nova-premier-v1:0:1000k, Name: Nova Premier
Model ID: amazon.nova-premier-v1:0:mm, Name: Nova Premier
Model ID: amazon.nova-premier-v1:0, Name: Nova Premier
Model ID: amazon.titan-text-premier-v1:0, Name: Titan Text G1 - Premier
Model ID: amazon.nova-pro-v1:0:24k, Name: Nova Pro
Model ID: amazon.nova-pro-v1:0:300k, Name: Nova Pro
Model ID: amazon.nova-pro-v1:0, Name: Nova Pro
Model ID: amazon.nova-lite-v1:0:24k, N

### Amazon Bedrock Cross-Region Inference
We will use Amazon Nova models for this example.  
  
For inference, we will be using the cross-region inference feature of Amazon Bedrock, which routes the inference call to the region which can best serve it at a given time.  
Cross-region inference [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html)  
For the latest model names using cross-region inference, refer to [Supported Regions and models](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles-support.html)  

**Important: Before using a model you need to be granted access to it from the AWS Management Console.**

In [ ]:
# Define the model and message
# Amazon Nova Pro is a multimodal inpu model - it can be prompted with images and text. We'll only be using text here.

QUESTION = ["I want you to help me pick a business area or industry that might be worth exploring for an Agentic AI opportunity.",
            "Expand on a pain point in that industry that is challenging and ready for an agentic AI solution.",
            "Based on that idea, describe a possible solution"]

BEDROCK_MODEL_ID = 'us.amazon.nova-pro-v1:0'

system = [{"text": "You are a helpful business consultant bot. You will respond in a professional, succinct tone in no more than 4 sentences."}]
for i in range(3):
    messages = [
        {"role": "user", "content": [{"text": QUESTION[i]}]}
        ]

    # Make the API call
    response = bedrock_runtime.converse(
        modelId=BEDROCK_MODEL_ID,
        messages=messages,
        system=system
    )

    # Store the response
    answer = response['output']['message']['content'][0]['text']

    # Store it into message history
    assistant_message = {"role": "assistant", "content":[{"text":answer}]}
    messages.append(assistant_message)
    print(f"{i}-Question: "+QUESTION[i]+"\nAnswer: " + answer)
    sleep(5) # Trying to be nice to avoid hitting rate limits

0-Question: I want you to help me pick a business area or industry that might be worth exploring for an Agentic AI opportunity.
Answer: Consider the healthcare sector for Agentic AI applications, focusing on patient management and personalized treatment plans.
1-Question: Expand on a pain point in that industry that is challenging and ready for an agentic AI solution.
Answer: Inefficient supply chain management causes delays and increased costs. Agentic AI can optimize logistics, predict demand, and automate inventory management, significantly enhancing operational efficiency.
2-Question: Based on that idea, describe a possible solution
Answer: Implement a customer feedback system to gather insights, analyze data for trends, and adjust strategies accordingly to enhance customer satisfaction and loyalty.
